## Data Analysis



- Get necessary stuff

In [7]:
import pandas as pd
import numpy as np
import os 
companies = ['zynga','supercell','king']
mainBranches = pd.read_csv("Assignment_1_Data/main_branches.csv",sep=';')

In [118]:
companies = ['Zynga','Supercell','King']
mainBranches = pd.read_csv("Assignment_1_Data/main_branches.csv",sep=';')
branches = mainBranches[mainBranches.company_name.isin(companies)]
#Collect all the subject Ids related to our companies
subjects = pd.read_csv("Assignment_1_Data/subjects_tree_data.csv",sep=';')
subjects.head()
def findValues(X):
    v = np.intersect1d(X[4].split(','),branches['branch_id'].values)
    if len(v) > 0:
        v = int(v[0])
        company = branches[branches.branch_id == v]['company_name'].values[0]
        prop = branches[branches.branch_id == v]['property'].values[0]
        return pd.Series({'subject_id': X[3],'branch_id':v,'subject':X[0],'property': prop,'company':company})
    else:
        return pd.Series()
subjects = subjects.apply(findValues,axis=1).dropna()
subjects.head()

,branch_id,company,property,subject,subject_id
10,1069691.0,Supercell,Boom Beach,Boom Beach,5073984.0
14,1013836.0,Zynga,Words With Friends,Words With Friends Pro,5338124.0
18,1013836.0,Zynga,Words With Friends,Words With Friends HD,5277124.0
19,1040093.0,Supercell,Clash Royale,Clash Royale,5450856.0
20,1069691.0,Supercell,Boom Beach,Boom Beach,5063087.0


#### First Phase

- a) What is the average monthly user growth rates of Supercell, King and Zynga?
- b) What is the net reach of Supercell, King and Zynga in the sample for each of the 3 months?
- c) What is the total time spent per month per user for Supercell, King and Zynga for July?
- d) Which of Supercell, King and Zynga had the largest average session duration in June?
    - All of these require session data
- e) What is the gender distribution of Supercell vs. King users?
     - This can be looked up from gender data


In [126]:
sessiondataDump = pd.read_csv("Assignment_1_Data/sessions_data.csv",sep=';')
sessionData = sessiondataDump[sessiondataDump.subject_id.isin(subjects.subject_id)]
del sessiondataDump
sessionData.head()
sessionData.head()
sessionData['start_timestamp'] = pd.to_datetime(sessionData['start_timestamp'])
sessionData['end_timestamp'] = pd.to_datetime(sessionData['end_timestamp'])

,subject_id,start_timestamp,end_timestamp,interaction_type,user_id,device_id,os
137,5144205,2016-05-06 13:33:38,2016-05-06 13:51:48,app,4530,10124,ios
138,5144205,2016-07-01 18:12:55,2016-07-01 18:23:28,app,4530,10124,ios
139,5144205,2016-06-05 23:53:40,2016-06-06 00:18:49,app,4530,10124,ios
140,5144205,2016-07-01 11:45:36,2016-07-01 11:56:09,app,4530,10124,ios
141,5144205,2016-06-02 14:57:03,2016-06-02 15:09:05,app,4530,10124,ios


In [138]:
sessionData['month'] = sessionDat

dtype('<M8[ns]')